In [1]:
import chromatix.functional as cf
import jax.numpy as jnp
from sample import single_bead_sample 
from jax import Array

import matplotlib.pyplot as plt
from tensor_tomo import thick_polarised_sample

%config InlineBackend.figure_format='retina'

In [10]:
def run_sim(orientation: Array) -> tuple[Array, Array]:
    # Our input field is a plane wave
    shape = (5 * 90, 5 * 90) # sample size
    spacing = 0.065 # from paper
    wavelength = 0.405 # from paper
    n_m = 1.33
    polarisation = cf.linear(0) # x-polarised light
    n_bead = jnp.array([1.44, 1.40, 1.37])  # z y x
    bead_radius = 1.5
    k0 = 2 * jnp.pi / wavelength  # mum
    shape = (4.55, 11.7 / 2, 11.7 / 2)  # z y x # / 2 as we have only 1 bead

    # Making incoming field
    in_field = cf.plane_wave(shape, spacing, wavelength, 1.0, amplitude=polarisation, scalar=False, pupil=lambda f: cf.square_pupil(f, w =90*spacing), power=100)
    
    # Making sample
    sample = single_bead_sample(n_m, n_bead, orientation, bead_radius, shape, spacing, k0)
    sample = cf.center_pad(sample, (0, 2 * 90, 2 * 90, 0, 0))    # Padding sample 

    # Running simulation
    field = thick_polarised_sample(in_field, sample, n_m, dz=spacing)

    # Making plotting a little easier
    # ::-1 to invert y axis
    amplitude = field.amplitude.squeeze()
    amplitude = amplitude[::-1][2 * 90:-2 * 90, 2 * 90:-2 * 90]

    phase = field.phase.squeeze()
    phase = phase[::-1][2 * 90:-2 * 90, 2 * 90:-2 * 90]

    return amplitude, phase

In [11]:
run_sim(jnp.array([0, 0, 0]))

TypeError: Shapes must be 1D sequences of concrete values of integer type, got (1, 4.55, 5.85, 5.85, 1, 3).